In [1]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

In [2]:
device = "cuda"
llm_model_path = '/home/yhchen/huggingface_model/Qwen/Qwen2-0.5B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(llm_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(llm_model_path, trust_remote_code=True).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 定义模型

主要就是重写 `_call` 函数：
- **定义指令：** 将输入的 prompt 添加到 message 中
- **定义模板：** 调用 tokenizer 的 apply_chat_template 函数，将 message 转换为 text
- **定义模型输入：** 将 text 转换为 model_inputs
- **定义模型输出：** 调用 model 的 generate 函数，解码并生成 response

In [14]:
class Qwen2_LLM(LLM):
    def __init__(self):
        super().__init__()
    
    def _call(self, prompt: str,stop: Optional[List[str]] = None, run_manager: Optional[CallbackManagerForLLMRun] = None) -> str:
        message = [
            {"role":"system", "content":"you are a helpful assistant."},
            {"role":"user", "content":prompt}
        ]
        
        text = tokenizer.apply_chat_template(
            conversation=message, 
            tokenize=False, 
            add_generation_prompt=True  # 源码没看到，但qwen官方是这么写
        )

        model_inputs = tokenizer(text, return_tensors="pt").to(device)

        generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=True)

        # 因为会根据 所有的模板内容进行 generation， 所以要去掉前面的（每一次输入的）模板
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return response
        

    @property
    def _llm_type(self) -> str:
        return "Qwen2-1.5B"

In [4]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
import torch

### 加载数据

In [5]:
# 前提 pip install unstructured
dataset_path = '/home/yhchen/huggingface_model/cfa532/CHLAWS'
loader = DirectoryLoader(dataset_path, glob='*.txt')
documents = loader.load()   # 将数据转换为 documents obj

### 数据切分 和 定义embeddings模型

- `RecursiveCharacterTextSplitter` 分割会更好

- `bge` 中文 embedding效果最好 （需要先安装 `pip install sentence_transformers`）

In [6]:
child_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
parent_spliter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)

embed_model_path = '/home/yhchen/huggingface_model/BAAI/bge-m3'
embeddings = HuggingFaceBgeEmbeddings(model_name=embed_model_path)

### 用 `ParentDocumentRetriever` 提高搜索质量

**父文档**存储在 `InMemoryStore` 中，**子文档**的embddings数据被存储在向量存储中。

如果不指定 `parent_spliter`。跟直接分割后存储到向量库的效果一致

In [7]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

persist_directory = 'chroma_data'
vectorstore = Chroma(collection_name="laws_document", embedding_function=embeddings, persist_directory=persist_directory)

store = InMemoryStore()

retriever = ParentDocumentRetriever(vectorstore=vectorstore, docstore=store, child_splitter=child_spliter, parent_splitter=parent_spliter)

/home/yhchen/anaconda3/envs/transformers/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


### 实例化 检索器 和 模型

In [8]:
retriever.add_documents(documents, id=None)

qwen = Qwen2_LLM()

In [15]:
qwen = Qwen2_LLM()
# chain 
from langchain import PromptTemplate

template = """
基于以下信息来回答用户问题。如果你不知道答案，就说你不知道，不要试图编造答案。尽量使答案简单，并最后回答的最后说“谢谢你的提问！”。                      
已知信息： 
{context} 
问题：
{question}
'''
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt":prompt}

qa = RetrievalQA.from_chain_type(
    llm=qwen, 
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
    )

In [16]:
result = qa.invoke({"query":"抢劫判几年？"})
print(result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'query': '抢劫判几年？', 'result': '根据法律规定，抢劫罪的量刑标准如下：\n\n1. 第一百六十三条：犯抢劫罪的，处三年以上十年以下有期徒刑，并处罚金；\n2. 第一百六十四条：犯抢劫罪的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产；\n\n3. 法律另有规定的，依照规定。\n\n所以，抢劫罪的最高刑期为10年，最低刑期为3年。', 'source_documents': [Document(metadata={'source': '/home/yhchen/huggingface_model/cfa532/CHLAWS/laws2023.txt'}, page_content='第五章 侵犯财产罪\n\n第二百六十三条 【抢劫罪】以暴力、胁迫或者其他方法抢劫公私财物的,处三年以上十年以下有期徒刑,并处罚金;有下列情形之一的,处十年以上有期徒刑、无期徒刑或者死刑,并处罚金或者没收财产:\n\n(一)入户抢劫的;\n\n(二)在公共交通工具上抢劫的;\n\n(三)抢劫银行或者其他金融机构的;\n\n(四)多次抢劫或者抢劫数额巨大的;\n\n(五)抢劫致人重伤、死亡的;\n\n(六)冒充军警人员抢劫的;\n\n(七)持枪抢劫的;\n\n(八)抢劫军用物资或者抢险、救灾、救济物资的。\n\n第二百六十四条 【盗窃罪】盗窃公私财物,数额较大的,或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的,处三年以下有期徒刑、拘役或者管制,并处或者单处罚金;数额巨大或者有其他严重情节的,处三年以上十年以下有期徒刑,并处罚金;数额特别巨大或者有其他特别严重情节的,处十年以上有期徒刑或者无期徒刑,并处罚金或者没收财产。\n\n第二百六十五条 【盗窃罪】以牟利为目的,盗接他人通信线路、复制他人电信码号或者明知是盗接、复制的电信设备、设施而使用的,依照本法第二百六十四条的规定定罪处罚。\n\n第二百六十六条 【诈骗罪】诈骗公私财物,数额较大的,处三年以下有期徒刑、拘役或者管制,并处或者单处罚金;数额巨大或者有其他严重情节的,处三年以上十年以下有期徒刑,并处罚金;数额特别巨大或者有其他特别严重情节的,处十年以上有期徒刑或者无期徒刑,并处罚金或者没收财产。本法另有规定的,依照规定。\n\n第二百六十七条 【抢夺罪】抢夺公私财物,数额较大的,或者多次抢夺的,处三年以下有

In [17]:
print(result['result'])

根据法律规定，抢劫罪的量刑标准如下：

1. 第一百六十三条：犯抢劫罪的，处三年以上十年以下有期徒刑，并处罚金；
2. 第一百六十四条：犯抢劫罪的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产；

3. 法律另有规定的，依照规定。

所以，抢劫罪的最高刑期为10年，最低刑期为3年。


In [19]:
result1 = qa.invoke({"query":"犯什么罪是无期徒刑而不是死刑"})
result1

{'query': '犯什么罪是无期徒刑而不是死刑',
 'result': '犯故意杀人罪是无期徒刑而不是死刑。',
 'source_documents': [Document(metadata={'source': '/home/yhchen/huggingface_model/cfa532/CHLAWS/laws2023.txt'}, page_content='40.塔吉克族              1名\n\n新疆维吾尔自治区1名\n\n41.怒族                1名\n\n云南省1名\n\n42.乌孜别克族             1名\n\n新疆维吾尔自治区1名\n\n43.俄罗斯族              1名\n\n新疆维吾尔自治区1名\n\n44.鄂温克族              1名\n\n内蒙古自治区1名\n\n45.德昂族               1名\n\n云南省1名\n\n46.保安族               1名\n\n甘肃省1名\n\n47.裕固族               1名\n\n甘肃省1名\n\n48.京族                1名\n\n广西壮族自治区1名\n\n49.塔塔尔族              1名\n\n新疆维吾尔自治区1名\n\n50.独龙族               1名\n\n云南省1名\n\n51.鄂伦春族              1名\n\n内蒙古自治区1名\n\n52.赫哲族               1名\n\n黑龙江省1名\n\n53.门巴族                1名\n\n西藏自治区1名\n\n54.珞巴族                1名\n\n西藏自治区1名\n\n55.基诺族               1名\n\n云南省1名\n\n二、中国人民解放军应选少数民族代表14名。\n\n三、其余26名少数民族代表名额由全国人民代表大会常务委员会依据法律另行分配。\n\n全国人民代表大会常务委员会关于授权国务院在粤港澳大湾区内地九市开展香港法律执业者和澳门执业律师取得内地执业资质和从事律师职业试点工作的决定\n\n全国人民代表大会常务委员会关于授权国务院在粤港澳大湾区内地九市开展香港法律执业者和澳门执业律师取得内地执业资质和从事律师

In [ ]:
print(result1["result"])

In [20]:
result2 = qa.invoke({"query":"走私判几年？有无期徒刑吗"})
result2

{'query': '走私判几年？有无期徒刑吗',
 'result': '走私被判刑的期限为3年到10年不等，最高不超过7年，但走私毒品罪和走私假币罪的判刑年限较长。走私毒品罪的判刑年限为15年到20年，走私假币罪的判刑年限为15年到20年。走私违禁品罪的判刑年限为5年到7年，走私枪支罪的判刑年限为10年到14年，走私军火罪的判刑年限为10年到15年，走私爆炸物罪的判刑年限为8年到15年，走私剧毒化学品罪的判刑年限为10年到15年。走私武器、弹药、核材料或者伪造的货币的，处七年以上有期徒刑，也可以并处窃取财物总额的百分之五十以上二倍以下罚金；对人体健康造成严重危害或者有其他严重情节的，处三年以上七年以下有期徒刑，可以并处窃取财物总额的百分之五十以上二倍以下罚金或者没收财产。走私国家禁止出口的文物、黄金、白银和其他贵重金属或者国家禁止进出口的珍贵动物及其制品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之五十以上二倍以下罚金；走私珍稀植物及其制品等国家禁止进出口的其他货物、物品的，处五年以下有期徒刑或者拘役，可以并处窃取财物总额的百分之二以上百分之五以下罚金。走私武器、弹药、核材料或者伪造的货币的，处五年以下有期徒刑或者拘役，可以并处窃取财物总额的百分之二以上百分之五以下罚金。走私普通货物、物品罪的，处三年以上七年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金；走私珍贵动植物及其制品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金；走私其他动植物及物品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金。走私淫秽物品罪的，处三年以上七年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金；走私珍贵动物及其制品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚',
 'source_documents': [Document(metadata={'source': '/home/yhchen/huggingface_model/cfa532/CHLAWS/laws2023.txt'}, page_content='第一百四十五条 【生产、销售不符合标准的医用器材罪】生产不符合保障人体健康的国家标准、行业标准的医疗器械、医用卫生材料,或

In [21]:
print(result2["result"])

走私被判刑的期限为3年到10年不等，最高不超过7年，但走私毒品罪和走私假币罪的判刑年限较长。走私毒品罪的判刑年限为15年到20年，走私假币罪的判刑年限为15年到20年。走私违禁品罪的判刑年限为5年到7年，走私枪支罪的判刑年限为10年到14年，走私军火罪的判刑年限为10年到15年，走私爆炸物罪的判刑年限为8年到15年，走私剧毒化学品罪的判刑年限为10年到15年。走私武器、弹药、核材料或者伪造的货币的，处七年以上有期徒刑，也可以并处窃取财物总额的百分之五十以上二倍以下罚金；对人体健康造成严重危害或者有其他严重情节的，处三年以上七年以下有期徒刑，可以并处窃取财物总额的百分之五十以上二倍以下罚金或者没收财产。走私国家禁止出口的文物、黄金、白银和其他贵重金属或者国家禁止进出口的珍贵动物及其制品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之五十以上二倍以下罚金；走私珍稀植物及其制品等国家禁止进出口的其他货物、物品的，处五年以下有期徒刑或者拘役，可以并处窃取财物总额的百分之二以上百分之五以下罚金。走私武器、弹药、核材料或者伪造的货币的，处五年以下有期徒刑或者拘役，可以并处窃取财物总额的百分之二以上百分之五以下罚金。走私普通货物、物品罪的，处三年以上七年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金；走私珍贵动植物及其制品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金；走私其他动植物及物品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金。走私淫秽物品罪的，处三年以上七年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚金；走私珍贵动物及其制品的，处五年以上十年以下有期徒刑，可以并处窃取财物总额的百分之二以上百分之五以下罚


In [22]:
result3 = qa.invoke({"query":"法律上怎么定义非法经营罪？"})
result3

{'query': '法律上怎么定义非法经营罪？',
 'result': '非法经营罪是指违反国家规定,有下列非法经营行为之一,扰乱市场秩序,情节严重的,处五年以下有期徒刑或者拘役,并处或者单处违法所得一倍以上五倍以下罚金;情节特别严重的,处五年以上有期徒刑,并处违法所得一倍以上五倍以下罚金或者没收财产:  \n(一)未经许可经营法律、行政法规规定的专营、专卖物品或者其他限制买卖的物品的;  \n(二)买卖进出口许可证、进出口原产地证明以及其他法律、行政法规规定的经营许可证或者批准文件的;  \n(三)未经国家有关主管部门批准非法经营证券、期货、保险业务的,或者非法从事资金支付结算业务的;  \n(四)其他严重扰乱市场秩序的非法经营行为。',
 'source_documents': [Document(metadata={'source': '/home/yhchen/huggingface_model/cfa532/CHLAWS/laws2023.txt'}, page_content='第二百二十五条 【非法经营罪】违反国家规定,有下列非法经营行为之一,扰乱市场秩序,情节严重的,处五年以下有期徒刑或者拘役,并处或者单处违法所得一倍以上五倍以下罚金;情节特别严重的,处五年以上有期徒刑,并处违法所得一倍以上五倍以下罚金或者没收财产:\n\n(一)未经许可经营法律、行政法规规定的专营、专卖物品或者其他限制买卖的物品的;\n\n(二)买卖进出口许可证、进出口原产地证明以及其他法律、行政法规规定的经营许可证或者批准文件的;\n\n(三)未经国家有关主管部门批准非法经营证券、期货、保险业务的,或者非法从事资金支付结算业务的;\n\n(四)其他严重扰乱市场秩序的非法经营行为。\n\n第二百二十六条 【强迫交易罪】以暴力、威胁手段,实施下列行为之一,情节严重的,处三年以下有期徒刑或者拘役,并处或者单处罚金;情节特别严重的,处三年以上七年以下有期徒刑,并处罚金:\n\n(一)强买强卖商品的;\n\n(二)强迫他人提供或者接受服务的;\n\n(三)强迫他人参与或者退出投标、拍卖的;\n\n(四)强迫他人转让或者收购公司、企业的股份、债券或者其他资产的;\n\n(五)强迫他人参与或者退出特定的经营活动的。\n\n第二百二十七条 【伪造、倒卖伪造的有价票证罪】伪造或者倒卖伪造的车票

In [23]:
print(result3["result"])

非法经营罪是指违反国家规定,有下列非法经营行为之一,扰乱市场秩序,情节严重的,处五年以下有期徒刑或者拘役,并处或者单处违法所得一倍以上五倍以下罚金;情节特别严重的,处五年以上有期徒刑,并处违法所得一倍以上五倍以下罚金或者没收财产:  
(一)未经许可经营法律、行政法规规定的专营、专卖物品或者其他限制买卖的物品的;  
(二)买卖进出口许可证、进出口原产地证明以及其他法律、行政法规规定的经营许可证或者批准文件的;  
(三)未经国家有关主管部门批准非法经营证券、期货、保险业务的,或者非法从事资金支付结算业务的;  
(四)其他严重扰乱市场秩序的非法经营行为。
